# Particle transport by a mean velocity field

We already defined and populated `U` and `V` fields (surface velocities averaged over ten years). <br>
In this module, we will use these velocity fields to transport particles across the globe.

Let us include everything we have done in the previous module and check that our building blocks are there (grid, U, and V). <br>
We can use the `NBInclude` package, which allows us to include notebooks.

In [1]:
using NBInclude 

@nbinclude("01_build_a_discrete_ocean.ipynb")

    Updating registry at `/srv/julia/pkg/registries/General.toml`
   Resolving package versions...
  No Changes to `/srv/julia/pkg/environments/v1.7/Project.toml`
  No Changes to `/srv/julia/pkg/environments/v1.7/Manifest.toml`


In [2]:
grid, U, V

(120×50×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo:
├── immersed_boundary: GridFittedBoundary{Field{Center, Center, Center, Nothing, LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded, OffsetArrays.OffsetVector{Float64, Vector{Float64}}, Float64, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, CPU}, Tuple{Colon, Colon, Colon}, OffsetArrays.OffsetArray{Bool, 3, Array{Bool, 3}}, Bool, FieldBoundaryConditions{BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditi

### Defining particles' initial positions

Particles in Oceananigans are identified by an x, y, and z position. (In the case of a spherical domain, x and y are longitude and latitude, respectively)

Initial positions are specified as arrays. <br> 
Let us distribute the initial positions around a Center with some random values:

$$\lambda_p = \lambda_c + S * (R - 0.5),$$

where $R$ is a random value between 0 and 1 and $S$ controls the spread of the initial positions

In [3]:
λ₀, φ₀ = -120.0, -55.0

spread_λ = 10.0
spread_φ = 10.0

n_particles = 200

# Arrays of uniformely distributed random numbers between 0 and 1 and built with the `rand(size)` function
λₚ = λ₀ .+ spread_λ .* (rand(n_particles) .- 0.5);
φₚ = φ₀ .+ spread_φ .* (rand(n_particles) .- 0.5);

# z is constant because the simulation is 2D
zₚ = 0.5 .* ones(n_particles);

# The function `extrema(array)` returns `(minimum(array), maximum(array))`
extrema(λₚ), extrema(φₚ)

((-124.94562921008206, -115.00586736845523), (-59.995980259034845, -50.02303641869135))

### Particles in Oceananigans

By passing the arrays containing the initial positions to the particles' <br>
constructor, we can build a `LagrangianParticles` object, the type which contains particles' <br>
properties in Oceananigans

In [4]:
lagrangian_particles = LagrangianParticles(x=λₚ, y=φₚ, z=zₚ)

200 LagrangianParticles with eltype Particle:
├── 3 properties: (:x, :y, :z)
├── particle-wall restitution coefficient: 1.0
├── 0 tracked fields: ()
└── dynamics: no_dynamics

### Oceananigans' Model type

A model in Oceananigans is a container that includes all the physical phenomena we want to represent. <br>

In our case, we only want to have particles transported by the velocity fields we previously defined. <br>

In [5]:
model = HydrostaticFreeSurfaceModel(grid = grid, velocities = PrescribedVelocityFields(u = U, v = V, w = W),
                                    buoyancy  = nothing,
                                    tracers   = (),
                                    particles = lagrangian_particles)

HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── grid: 120×50×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: ()
├── closure: Nothing
├── buoyancy: Nothing
├── coriolis: Nothing
└── particles: 200 Lagrangian particles with 3 properties: (:x, :y, :z)

### Time step size

As a grid discretizes space, the time step "discretizes" the evolution in time. <br>
The time step size is a fundamental parameter of the simulation and defines the resolution in time. <br>
as a rule of thumb, smaller time steps -> a more stable simulation!

#### CFL condition

In a time step of $\Delta t$ a particle with velocity $V$ will traverse a distance of $V \cdot \Delta t$. <br>
For fidelity reasons, particle should not traverse more than one grid cell in one time step. <br>
If the cell size is $\Delta x$, there is a maximum $\Delta t$ that satisfies the above condition, 

$$\Delta t_{max} = \frac{\Delta x}{V} \ ,$$

We can rewrite this condition as so:

$$\text{CFL} = V \cdot \frac{\Delta t}{\Delta x} \le 1  \ .$$

This inequality is commonly called the Courant-Friedrichs-Lewy (CFL) condition where the left-hand side is called the "CFL" number. <br> (https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) <br>

In Oceananigans, it is possible to check the maximum $\Delta t$  with the `CFL` type that can be "used" as a function.

Let us try a time step of 6 hours which is reasonable for oceanic time scales

In [6]:
Δt  = 6hours

CFL(Δt)(model)

0.1788495238557499

Looking at the CFL number is the $\Delta t$ we chose appropriate? <br>
Can we increase it / Do we have to decrease it?

### Defining a simulation

The last step is to define a Simulation. <br>
A Simulation type in Oceananigans is a wrapper around the model, which include <br>
all the additional technical details of the simulation:
- the time step.
- stopping conditions (either `stop_time` or `stop_iteration`)
- `Callbacks` or custom functions to be called during the simulation

In [7]:
simulation = Simulation(model, Δt = Δt, stop_time = 10years);

start_time = [time_ns()]

simulation.callbacks[:progress] = Callback(progress, IterationInterval(500));

# We write particles every 10 simulation days into the empty output arrays
particles_output = []
simulation.callbacks[:save_output] = Callback(store_particles_output!, TimeInterval(10days))

simulation

Simulation of HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 hours
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 10 years
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 6 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   ├── progress => Callback of progress on IterationInterval(500)
│   └── save_output => Callback of store_particles_output! on TimeInterval(10 days)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

### Let's run our first simulation!

All the necessary ingredients are there, time to run the simulation!

In [8]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/W63bs/src/Simulations/run.jl:167
┌ Info: Time:    0 seconds, iteration: 0, wall time: 11.764 seconds
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:125
┌ Info:     ... simulation initialization complete (996.051 ms)
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/W63bs/src/Simulations/run.jl:202
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/W63bs/src/Simulations/run.jl:112
┌ Info:     ... initial time step complete (2.565 seconds).
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/W63bs/src/Simulations/run.jl:119
┌ Info: Time:     125 days, iteration: 500, wall time: 2.928 seconds
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:125
┌ Info: Time:     250 days, iteration: 1000, wall time: 38.214 ms
└ @ Main /home/jovyan/coessin

### Visualize the output

A visualization function for the output called `visualize_results(grid)` is already implemented in "visualize_particles.jl". <br>
(If you want to take a look, go ahead! Ask if you want to know how it works)

In [9]:
include("utils/visualize_particles.jl")

visualize_mean_velocity_results(model)

┌ Info: Plotting iteration 50 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 100 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 150 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 200 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 250 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 300 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: Plotting iteration 350 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:57
┌ Info: video recorded in output_particles.mp4
└ @ Main /home/jovyan/coessing2022-hub/julia

In [10]:
display_mp4("output_particles.mp4")

# Exercises

Now that we know all the ingredients to run a simple particle tracking simulation, we can <br>
try playing around with them to see how they affect particle tracks.

## (1) Check the influence of the initial position

Try changing the particle parameters and see where the particles end up
- initial position 
- spread
- number of particles

We should see particles accumulate in "convergence" regions <br>
These are regions where velocity is low, and the flow moves _downwards_. <br>
Particles are _buoyant_ (they float) and cannot follow the flow in <br>
its descent, thus accumulating on the ocean surface. <br>
Some interesting positions to try are:

***Southern Ocean*** <br>
λ₀, φ₀ = -120.0, -60.0

***Equatorial Pacific Ocean*** <br>
λ₀, φ₀ = -160.0, 0.0

***West Africa Coast*** <br>
λ₀, φ₀ = 55.2, 8.3

***Gulf stream (north Atlantic)*** <br>
λ₀, φ₀ = -75.0, 30.5

## (2) Check the influence of a coarser grid

As we learned, the number of cells ("pixels") in a grid is directly correlated to the "quality" <br>
of the simulation. <br>

Try coarsening/refining the grid (remember: the maximum resolution is 1440 X 600) and see the impact on particle trajectories.

## (3) Change the velocity fields

Try modifying the velocity fields (`U` and `V`) to see the impact of the currents on particle trajectory. <br>
Try reducing/increasing the velocity or changing the sign to see the flow moving in the other direction.

Otherwise, try increasing/decreasing the magnitude of the `V` velocity when compared to the `U` velocity <br>
to see particles move predominantly in the latitude/longitude direction or swap `U` and `V` to see new convergent zones arising

## Compare to observations? (You need turbulent velocities!!) 

Since plastics are a _buoyant_ tracer (they float so they "feel" only the surface velocity field), <br> 
the drift of plastics in the ocean generally follows the same dynamics we simulated here.

We could compare the results of the simulations with observations of plastic accumulation, but the surface of the ocean is <br>
generally turbulent (not time independent as we imposed here), so there might be significant discrepancies! <br>
If you want to do some realistic comparison you can look at `03_particles_transport_by_turbulent_currents_optional.ipynb`